# 🚀 Safety Vision AI - Entrenamiento en Google Colab
## YOLOv8 Fine-Tuning para Detección de EPP

**Dataset:** Hard Hat Detection (5,269 imágenes)
**Clases:** head, helmet, person
**Tiempo estimado con GPU T4:** 40-80 minutos

---

### 📋 INSTRUCCIONES ANTES DE EMPEZAR:

1. **Activar GPU:**
   - Menu: `Runtime` → `Change runtime type`
   - Hardware accelerator: **T4 GPU**
   - Clic en **Save**

2. **Preparar dataset:**
   - Necesitas subir el archivo ZIP del dataset
   - O tenerlo en Google Drive

3. **Ejecutar celdas en orden:**
   - Presiona `Shift + Enter` en cada celda
   - O usa: `Runtime` → `Run all`

## 1️⃣ Verificar GPU y Instalar Ultralytics

In [ ]:
# Instalar Ultralytics
!pip install ultralytics -q

# Verificar GPU
import torch
print("="*60)
print("🔥 VERIFICACIÓN DE GPU")
print("="*60)
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print("\n🚀 ¡PERFECTO! Entrenarás con GPU")
else:
    print("\n⚠️  WARNING: No GPU detectada")
    print("Ve a: Runtime → Change runtime type → T4 GPU")

## 2️⃣ Montar Google Drive (Opción A - RECOMENDADA)

**Usa esta opción si subiste el dataset a Google Drive**

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive montado")
print("\n📁 Ahora sube tu carpeta 'helmet_vest_detection' a:")
print("   /content/drive/MyDrive/Safety-Vision-AI/")

## 2️⃣-B Subir ZIP directamente (Opción B - Alternativa)

**Usa esta opción si tienes el dataset en un archivo ZIP**

**IMPORTANTE:** Primero comprime tu dataset localmente:
```powershell
# En tu PC, ejecuta:
cd C:\Users\LENOVO\OneDrive\Documentos\SISTEMASCORE\PROYECTOS\Safety-Vision-AI\datasets
Compress-Archive -Path "helmet_vest_detection\*" -DestinationPath "helmet_vest_detection.zip"
```

In [ ]:
# SOLO SI USAS OPCIÓN B (subir ZIP)
from google.colab import files
import zipfile

print("📤 Sube el archivo helmet_vest_detection.zip")
print("⏰ Esto tomará 5-10 minutos dependiendo de tu conexión...\n")

uploaded = files.upload()

# Descomprimir
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"\n📦 Descomprimiendo {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('dataset')
        print("✅ Dataset descomprimido en /content/dataset/")

## 3️⃣ Configurar Paths del Dataset

In [ ]:
from pathlib import Path

# ⚠️ ELIGE UNA OPCIÓN:

# OPCIÓN A: Si usaste Google Drive
dataset_path = Path('/content/drive/MyDrive/Safety-Vision-AI/helmet_vest_detection')

# OPCIÓN B: Si subiste ZIP directamente
# dataset_path = Path('/content/dataset')

# Verificar que existe
if not dataset_path.exists():
    print(f"❌ ERROR: No se encontró el dataset en {dataset_path}")
    print("\nVerifica la ruta o sube el dataset primero")
else:
    print(f"✅ Dataset encontrado en: {dataset_path}")
    
    # Contar imágenes
    train_images = list((dataset_path / 'train' / 'images').glob('*.jpg'))
    test_images = list((dataset_path / 'test' / 'images').glob('*.jpg'))
    
    print(f"\n📊 ESTADÍSTICAS:")
    print(f"   🔹 Imágenes de entrenamiento: {len(train_images):,}")
    print(f"   🔹 Imágenes de validación: {len(test_images):,}")
    print(f"   🔹 Total: {len(train_images) + len(test_images):,}")

## 4️⃣ Crear archivo data.yaml

In [ ]:
# Crear data.yaml
yaml_content = f"""
path: {dataset_path}
train: train/images
val: test/images

nc: 3
names: ['head', 'helmet', 'person']
"""

yaml_path = Path('/content/data.yaml')
yaml_path.write_text(yaml_content)

print("✅ data.yaml creado:")
print(yaml_content)

## 5️⃣ 🔥 ENTRENAR EL MODELO

### ⏰ Tiempo estimado: 40-80 minutos con GPU T4

**IMPORTANTE:** 
- No cierres esta pestaña mientras entrena
- Puedes ver el progreso en tiempo real
- Si se interrumpe, puedes reanudar desde el checkpoint

In [ ]:
from ultralytics import YOLO
from datetime import datetime

print("="*70)
print("🚀 INICIANDO ENTRENAMIENTO")
print("="*70)

start_time = datetime.now()
print(f"\n⏰ Inicio: {start_time.strftime('%Y-%m-%d %H:%M:%S')}\n")

# Cargar modelo base
model = YOLO('yolov8n.pt')

# Configuración de entrenamiento
results = model.train(
    data='/content/data.yaml',
    epochs=50,              # 50 epochs = excelentes resultados
    imgsz=640,              # Tamaño de imagen
    batch=32,               # GPU puede manejar batch más grande
    device=0,               # GPU
    project='/content/runs',
    name='helmet_detection',
    patience=10,            # Early stopping
    save=True,
    plots=True,
    
    # Data augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10.0,
    translate=0.1,
    scale=0.5,
    flipud=0.0,
    fliplr=0.5,
)

end_time = datetime.now()
duration = end_time - start_time

print("\n" + "="*70)
print("🎉 ¡ENTRENAMIENTO COMPLETADO!")
print("="*70)
print(f"\n⏰ Duración total: {duration}")
print(f"⏰ Finalizado: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")

## 6️⃣ Evaluar el Modelo

In [ ]:
# Evaluar en conjunto de validación
print("\n📊 EVALUANDO MODELO...\n")

metrics = model.val()

print("\n" + "="*70)
print("📈 MÉTRICAS FINALES")
print("="*70)
print(f"\n🎯 mAP@0.5:        {metrics.box.map50:.4f}")
print(f"🎯 mAP@0.5:0.95:   {metrics.box.map:.4f}")
print(f"🎯 Precision:      {metrics.box.mp:.4f}")
print(f"🎯 Recall:         {metrics.box.mr:.4f}")

# Verificar objetivo
if metrics.box.map50 >= 0.75:
    print("\n✅ ¡EXCELENTE! El modelo cumple el objetivo (mAP@0.5 >= 0.75)")
else:
    print(f"\n⚠️  mAP@0.5 = {metrics.box.map50:.4f} (objetivo: >= 0.75)")
    print("Considera entrenar más epochs o ajustar hiperparámetros")

## 7️⃣ Visualizar Curvas de Entrenamiento

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

# Mostrar gráfica de resultados
results_img = Path('/content/runs/helmet_detection/results.png')

if results_img.exists():
    img = Image.open(results_img)
    plt.figure(figsize=(16, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Curvas de Entrenamiento', fontsize=18, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Gráficas mostradas correctamente")
else:
    print("⚠️  No se encontró la gráfica de resultados")

## 8️⃣ Probar el Modelo en Imágenes

In [ ]:
import random

# Cargar el mejor modelo
best_model_path = '/content/runs/helmet_detection/weights/best.pt'
best_model = YOLO(best_model_path)

# Seleccionar imágenes aleatorias de validación
test_images_list = list((dataset_path / 'test' / 'images').glob('*.jpg'))
sample_images = random.sample(test_images_list, min(6, len(test_images_list)))

print("🎯 PREDICCIONES EN IMÁGENES DE VALIDACIÓN:\n")

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, img_path in enumerate(sample_images):
    # Hacer predicción
    results = best_model(str(img_path), verbose=False)
    
    # Dibujar predicciones
    annotated = results[0].plot()
    
    # Mostrar
    axes[idx].imshow(annotated[..., ::-1])  # BGR to RGB
    axes[idx].set_title(f"Predicción {idx+1}", fontsize=14, fontweight='bold')
    axes[idx].axis('off')
    
    # Imprimir detecciones
    print(f"Imagen {idx+1}: {img_path.name}")
    for box in results[0].boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        name = results[0].names[cls]
        print(f"  ➤ {name}: {conf:.2f}")
    print()

plt.tight_layout()
plt.suptitle('Predicciones del Modelo Entrenado', fontsize=18, fontweight='bold', y=1.02)
plt.show()

## 9️⃣ 💾 DESCARGAR EL MODELO ENTRENADO

**IMPORTANTE:** Descarga estos archivos antes de cerrar Colab

In [ ]:
from google.colab import files
import shutil

print("📥 DESCARGANDO ARCHIVOS...\n")

# 1. Copiar el mejor modelo con nombre descriptivo
best_model = Path('/content/runs/helmet_detection/weights/best.pt')
if best_model.exists():
    # Renombrar con información de métricas
    model_name = f'yolov8_helmet_vest_best_mAP{metrics.box.map50:.3f}.pt'
    shutil.copy(best_model, f'/content/{model_name}')
    
    print(f"1️⃣ Descargando modelo entrenado: {model_name}")
    files.download(f'/content/{model_name}')
    print(f"   ✅ Modelo descargado ({best_model.stat().st_size / (1024*1024):.2f} MB)\n")

# 2. Descargar gráficas de entrenamiento
results_png = Path('/content/runs/helmet_detection/results.png')
if results_png.exists():
    shutil.copy(results_png, '/content/training_results.png')
    print("2️⃣ Descargando gráficas de entrenamiento")
    files.download('/content/training_results.png')
    print("   ✅ Gráficas descargadas\n")

# 3. Descargar matriz de confusión
confusion_matrix = Path('/content/runs/helmet_detection/confusion_matrix.png')
if confusion_matrix.exists():
    shutil.copy(confusion_matrix, '/content/confusion_matrix.png')
    print("3️⃣ Descargando matriz de confusión")
    files.download('/content/confusion_matrix.png')
    print("   ✅ Matriz de confusión descargada\n")

print("="*70)
print("✅ TODOS LOS ARCHIVOS DESCARGADOS")
print("="*70)
print("\n📁 Guarda estos archivos en tu proyecto local:")
print("   models_assets/yolov8_helmet_vest_best.pt")

## 🔟 Resumen Final y Próximos Pasos

In [ ]:
print("\n" + "="*70)
print("🎉 FASE 1 COMPLETADA - FINE-TUNING EXITOSO")
print("="*70)

print("\n📊 RESUMEN:")
print(f"   🔹 Dataset: {len(train_images):,} imágenes de entrenamiento")
print(f"   🔹 Modelo: YOLOv8n")
print(f"   🔹 Epochs: 50")
print(f"   🔹 Duración: {duration}")
print(f"   🔹 mAP@0.5: {metrics.box.map50:.4f}")
print(f"   🔹 Precision: {metrics.box.mp:.4f}")
print(f"   🔹 Recall: {metrics.box.mr:.4f}")

print("\n✅ ARCHIVOS DESCARGADOS:")
print(f"   📁 {model_name}")
print(f"   📁 training_results.png")
print(f"   📁 confusion_matrix.png")

print("\n🚀 PRÓXIMOS PASOS EN TU PC:")
print("\n1️⃣ Mover el modelo a tu proyecto:")
print("   - Copia el archivo .pt descargado a:")
print("     C:\\Users\\LENOVO\\OneDrive\\Documentos\\SISTEMASCORE\\PROYECTOS\\")
print("     Safety-Vision-AI\\models_assets\\yolov8_helmet_vest_best.pt")

print("\n2️⃣ Commitear a GitHub:")
print("   git add models_assets/yolov8_helmet_vest_best.pt")
print(f"   git commit -m 'feat: train YOLOv8 model with mAP={metrics.box.map50:.3f}'")
print("   git push")

print("\n3️⃣ Continuar con FASE 2:")
print("   - Pipeline de Inferencia en Tiempo Real")
print("   - Detección en video")
print("   - Lógica de seguridad")

print("\n" + "="*70)
print("🎊 ¡EXCELENTE TRABAJO MANITO! 🎊")
print("="*70)